# Exercícios
Analise o dataset Diabetic Retinopathy e estabeleça um protocolo de execução.  
* Quantas classes existem neste dataset?  
* Como podemos carregar as imagens para a execução?  
* Adicionalmente, após a formação do conjunto, compare a performance dos vetores obtidos na VGG com a ResNet em 3 classificadores diferentes.  

As classes do dataset estão divididas em pastas, as 5 classes são:  
Healthy  
Mild DR  
Moderate DR  
Proliferate DR  
Severe DR

As imagens foram hospedadas num repositorio github e então carregados no google colab

In [ ]:
!git clone https://github.com/aaasatoou/BlueEdTech_M3.git

Cloning into 'BlueEdTech_M3'...
remote: Enumerating objects: 2657, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 2657 (delta 0), reused 3 (delta 0), pack-reused 2654
Receiving objects: 100% (2657/2657), 336.23 MiB | 16.81 MiB/s, done.


A biblioteca 'os' pode ser utilizada para acessar os arquivos carregados para que sejam transformados da forma mais adequada.

In [ ]:
import os
import numpy as np

from PIL import Image

In [ ]:
Healthy_path = '/content/BlueEdTech_M3/Datasets/Diabetic Retinopathy Dataset/Healthy/'
Mild_path = '/content/BlueEdTech_M3/Datasets/Diabetic Retinopathy Dataset/Mild DR/'
Moderate_path = '/content/BlueEdTech_M3/Datasets/Diabetic Retinopathy Dataset/Moderate DR/'
Proliferate_path = '/content/BlueEdTech_M3/Datasets/Diabetic Retinopathy Dataset/Proliferate DR/'
Severe_path = '/content/BlueEdTech_M3/Datasets/Diabetic Retinopathy Dataset/Severe DR/'

Healthy = os.listdir(Healthy_path)
Mild = os.listdir(Mild_path)
Moderate = os.listdir(Moderate_path)
Proliferate = os.listdir(Proliferate_path)
Severe = os.listdir(Severe_path)

paths = [Healthy_path, Mild_path, Moderate_path, Proliferate_path, Severe_path]
files = [Healthy, Mild, Moderate, Proliferate, Severe]

In [ ]:
X = np.empty(shape=(2750, 224, 224, 3)) # 2750 imagens. 224 x 224 será o tamanho da imagem, em 3 canais (RGB)
y = []
contagem = 0

for i in range(0, len(paths)):
  for j in files[i]:
    a = Image.open(paths[i] + j).convert('RGB').resize(size=(224,224))
    X[contagem] = np.array(a)
    y.append(paths[i][61:-1])
    contagem += 1

X = X/255

Com o conjunto adequado, basta obter as características utilizando as redes neurais já treinadas e então classificá-los.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications import resnet50, VGG19

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import lightgbm as lgb

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= 95)

print("Loading the ResNet50-ImageNet model ...")
model = resnet50.ResNet50(include_top=True, weights='imagenet', input_shape=(224, 224, 3), classes=1000)
model = Model(inputs=model.input, outputs=model.get_layer(index=-2).output)
#model.summary()

print('\tFeatures training shape: ', X_train.shape)
print('\tFeatures testing shape: ', X_test.shape)
print('\nFeature extraction...\n')
prediction = np.array(model.predict(X_train))
X_train = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

prediction = np.array(model.predict(X_test))
X_test = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

print('\tFeatures training shape: ', X_train.shape)
print('\tFeatures testing shape: ', X_test.shape)

Loading the ResNet50-ImageNet model ...
102981632/102967424 [==============================] - 1s 0us/step
	Features training shape:  (2200, 224, 224, 3)
	Features testing shape:  (550, 224, 224, 3)

Feature extraction...

	Features training shape:  (2200, 2048)
	Features testing shape:  (550, 2048)


In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
svc = SVC()
lgbm = lgb.LGBMClassifier()

knn.fit(X_train, np.ravel(y_train, order='C'))
svc.fit(X_train, np.ravel(y_train, order='C'))
lgbm.fit(X_train, np.ravel(y_train, order='C'))

result_knn = knn.predict(X_test)
result_svc = svc.predict(X_test)
result_lgbm = lgbm.predict(X_test)

acc_knn = accuracy_score(result_knn, y_test)
acc_svc = accuracy_score(result_svc, y_test)
acc_lgbm = accuracy_score(result_lgbm, y_test)

print("\tAccuracy Linear KNN: %0.4f" % acc_knn)
print("\tAccuracy SVM: %0.4f" % acc_svc)
print("\tAccuracy LGBM: %0.4f" % acc_lgbm)

	Accuracy Linear KNN: 0.6382
	Accuracy SVM: 0.5982
	Accuracy LGBM: 0.6818


In [ ]:
from keras.backend import clear_session

clear_session() # Limpa o modelo anterior

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= 95)

# Limpar essas variáveis libera espaço na memória, como não serão mais utilizadas, devem ser deletadas para que seja possível a execução no google colab.
del X 
del y

model = VGG19(include_top=True, weights='imagenet', input_shape=(224, 224, 3), classes=1000)
model = Model(inputs=model.input, outputs=model.get_layer(index=-2).output)
# model.summary()

print('\tFeatures training shape: ', X_train.shape)
print('\tFeatures testing shape: ', X_test.shape)
print('\nFeature extraction...\n')
prediction = np.array(model.predict(X_train))
X_train = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

prediction = np.array(model.predict(X_test))
X_test = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

print('\tFeatures training shape: ', X_train.shape)
print('\tFeatures testing shape: ', X_test.shape)

574726144/574710816 [==============================] - 16s 0us/step
	Features training shape:  (2200, 224, 224, 3)
	Features testing shape:  (550, 224, 224, 3)

Feature extraction...

	Features training shape:  (2200, 4096)
	Features testing shape:  (550, 4096)


In [ ]:
knn.fit(X_train, np.ravel(y_train, order='C'))
svc.fit(X_train, np.ravel(y_train, order='C'))
lgbm.fit(X_train, np.ravel(y_train, order='C'))

result_knn = knn.predict(X_test)
result_svc = svc.predict(X_test)
result_lgbm = lgbm.predict(X_test)

acc_knn = accuracy_score(result_knn, y_test)
acc_svc = accuracy_score(result_svc, y_test)
acc_lgbm = accuracy_score(result_lgbm, y_test)

print("\tAccuracy Linear KNN: %0.4f" % acc_knn)
print("\tAccuracy SVM: %0.4f" % acc_svc)
print("\tAccuracy LGBM: %0.4f" % acc_lgbm)

	Accuracy Linear KNN: 0.6345
	Accuracy SVM: 0.6273
	Accuracy LGBM: 0.6818


As performances são bem semelhantes, as métricas obtidas pelos vetores extraídos da VGG19 são levemente melhores.